In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

from tensorflow import feature_column as fc

import tensorflow as tf

### Loading Data

In [2]:
# loading dataset
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # trainig data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

CATEGORICAL_COLUMNS = ["sex", "n_siblings_spouses", "class", "parch", "deck", "embark_town", "alone"]
NUMERICAL_COLUMNS = ["age", "fare"]

In [3]:
feature_coulumns = []

for feature_name in CATEGORICAL_COLUMNS:
    vocabulray = dftrain[feature_name].unique()
    feature_coulumns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulray))

for feature_name in NUMERICAL_COLUMNS:
    feature_coulumns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [4]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

# Training model

In [5]:
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_coulumns, model_dir="./models_regression")
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])

0.780303


In [6]:
result = list(linear_est.predict(eval_input_fn))
clear_output()
print(dfeval.loc[4])
print(y_eval.loc[4])
print(result[4]['probabilities'][1])

sex                          male
age                            34
n_siblings_spouses              0
parch                           0
fare                           13
class                      Second
deck                            D
embark_town           Southampton
alone                           y
Name: 4, dtype: object
1
0.21879567
